Explore daily case counts of Dengue fever in São Paulo from 2013 through 2020, indexed by symptom onset (DT_SIN_PRI) and reporting date (DT_NOTIFIC), (accessed from https://zenodo.org/records/15292499 at 27/05/2025)

In [1]:
import pandas as pd

start_year = 2013
end_year = 2020
max_delay = 40

path = "../data/DENGSP.csv"
dengdf_raw = pd.read_csv(path, index_col=0)
print(dengdf_raw.head())


/var/folders/5f/9g0vtmkj22j_xbhxyrknnv140000gn/T/ipykernel_96057/3295929074.py:8: DtypeWarning: Columns (7,11,23,45,46,47,55,65,69,75,86,102) have mixed types. Specify dtype option on import or set low_memory=False.
  dengdf_raw = pd.read_csv(path, index_col=0)


   TP_NOT ID_AGRAVO  DT_NOTIFIC  SEM_NOT  NU_ANO  SG_UF_NOT ID_MUNICIP  \
0       2       A90  2013-01-27   201305    2013         12     120040   
1       2       A90  2013-02-03   201306    2013         12     120010   
2       2       A90  2013-06-18   201325    2013         27     270030   
3       2       A90  2013-09-26   201339    2013         27     270030   
4       2       A90  2013-12-13   201350    2013         27     270030   

   ID_REGIONA  ID_UNIDADE  DT_SIN_PRI  ... LACO_N PLASMATICO  EVIDENCIA  \
0      1938.0   6439837.0  2013-01-25  ...    NaN        NaN        NaN   
1      1937.0   2001500.0  2013-01-28  ...    NaN        NaN        NaN   
2      1537.0   2005050.0  2013-06-17  ...    NaN        NaN        NaN   
3      1537.0   2005050.0  2013-09-18  ...    NaN        NaN        NaN   
4      1537.0   2005050.0  2013-12-11  ...    NaN        NaN        NaN   

  PLAQ_MENOR  CON_FHD  COMPLICA  TP_SISTEMA  NDUPLIC_N  CS_FLXRET  FLXRECEBI  
0        NaN      NaN    

In [2]:
# Filter for date of symptom onset and the reporting date
dengdf = dengdf_raw[['DT_SIN_PRI', 'DT_NOTIFIC']].dropna()
dengdf.columns = ['Date_Symptoms', 'Date_Reported']
dengdf = dengdf.apply(pd.to_datetime).sort_values(by='Date_Symptoms')

# Compute delay and convert to integer rather than Timedelta object
dengdf['Delay'] = (dengdf['Date_Reported'] - dengdf['Date_Symptoms']).dt.days
dengdf

,Date_Symptoms,Date_Reported,Delay
874476,1911-07-11,2015-06-01,37946
1966386,1926-05-10,2016-05-03,32866
416329,1928-04-10,2014-02-26,31368
1575892,1929-01-14,2015-06-21,31569
3296419,1929-02-28,2020-03-03,33241
...,...,...,...
2275510,2106-04-17,2016-04-21,-32867
2273005,2106-04-18,2016-04-22,-32867
2273641,2106-04-25,2016-04-26,-32870
2274616,2106-04-28,2016-05-02,-32867


In [3]:
# Filter between start and end year
dengdf = dengdf[
    (dengdf["Date_Symptoms"].dt.year >= start_year) & 
    (dengdf["Date_Symptoms"].dt.year <= end_year) &
    (dengdf["Date_Reported"].dt.year >= start_year) &
    (dengdf["Date_Reported"].dt.year <= end_year)
    ]
dengdf

,Date_Symptoms,Date_Reported,Delay
5688,2013-01-01,2013-01-08,7
9040,2013-01-01,2013-01-01,0
726,2013-01-01,2013-01-02,1
1808,2013-01-01,2013-01-03,2
9865,2013-01-01,2013-01-14,13
...,...,...,...
3184609,2020-12-31,2020-12-31,0
3557041,2020-12-31,2020-12-31,0
3455983,2020-12-31,2020-12-31,0
3539011,2020-12-31,2020-12-31,0


In [4]:
# Filter rows with delays greater tha max_delay
dengdf = dengdf[
    (dengdf['Delay'] < max_delay) &
    (dengdf['Delay'] >= 0)
    ]
dengdf

# Now want to create contingency table for every symptom date for each delay 
deng_delays = pd.crosstab(dengdf['Date_Symptoms'], dengdf['Delay'])
deng_delays

Delay,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
Date_Symptoms,,,,,,,,,,,,,,,,,,,,,
2013-01-01,12,57,58,74,22,6,62,45,31,17,...,2,3,0,0,1,1,0,0,0,0
2013-01-02,19,66,56,24,16,65,47,48,28,15,...,1,1,1,2,2,1,2,1,0,0
2013-01-03,29,79,28,20,73,47,52,43,23,1,...,0,0,3,2,0,1,0,0,0,0
2013-01-04,34,54,23,92,53,48,50,41,3,2,...,0,3,0,1,0,0,0,0,0,0
2013-01-05,28,36,68,61,59,44,46,3,2,30,...,2,2,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,14,43,50,26,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-12-28,31,65,34,19,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-12-29,16,35,25,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Check for any NAs across all (Delay,Symptom combinations)
(deng_delays.isna().sum() > 0).sum()

# Ensure all days between start_year and end_year are present
all_days = pd.DataFrame({
    "Date_Symptoms": pd.date_range(
        start=f"{start_year}-01-01", end=f"{end_year}-12-31")
})
deng_delays = all_days.merge(deng_delays, on="Date_Symptoms", how="left")
deng_delays.fillna(0).astype(int)

# Rename cols for clarity
deng_delays.columns = [deng_delays.columns[0]] + [f"delay_{col}" for col in deng_delays.columns[1:]]

deng_delays.to_csv('../data/dengue-sp-reporting-delay.csv', index=False)

deng_delays

,Date_Symptoms,delay_0,delay_1,delay_2,delay_3,delay_4,delay_5,delay_6,delay_7,delay_8,...,delay_30,delay_31,delay_32,delay_33,delay_34,delay_35,delay_36,delay_37,delay_38,delay_39
0,2013-01-01,12,57,58,74,22,6,62,45,31,...,2,3,0,0,1,1,0,0,0,0
1,2013-01-02,19,66,56,24,16,65,47,48,28,...,1,1,1,2,2,1,2,1,0,0
2,2013-01-03,29,79,28,20,73,47,52,43,23,...,0,0,3,2,0,1,0,0,0,0
3,2013-01-04,34,54,23,92,53,48,50,41,3,...,0,3,0,1,0,0,0,0,0,0
4,2013-01-05,28,36,68,61,59,44,46,3,2,...,2,2,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2917,2020-12-27,14,43,50,26,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2918,2020-12-28,31,65,34,19,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2919,2020-12-29,16,35,25,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2920,2020-12-30,13,23,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
